# 1D-transformer

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import ecg_transformer
import ecg_dataset
import pandas as pd
from torch.utils.data import DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # set to gpu if possible

In [ ]:
batch_size = 64 # input batch size for training (default: 64)
epochs = 25 # number of epochs to train (default: 10)
lr = 0.0001  # learning rate (default: 0.01)
emsize = 64 # embedding dimension == d_model
dim_feedforward = 256 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4 # the number of heads in the multiheadattention models
n_conv_layers = 2 # number of convolutional layers (before transformer encoder)
dropout = 0.25 # the dropout value
dropout_other = 0.1 # dropout value for feedforward output layers
n_class = 2 # not actually used atm

In [ ]:
model = ecg_transformer.TransformerModel(emsize, nhead, dim_feedforward, nlayers, n_conv_layers, n_class, dropout, dropout_other).to(device)

In [ ]:
model.train(mode=True) # Turn on the train mode

In [ ]:
meta_data = pd.read_parquet('../../data/freeze/BROAD_ml4h_klarqvist___physionet__meta_data__graded_splits__84fe7e5e413d4dc8b6de645ed5f06c5d.pq')
meta_data

In [ ]:
meta_data['Age'] = meta_data['Age'].astype(np.float32)
meta_data = meta_data[meta_data['n_observations'] >= 5000]

In [ ]:
h5py_path = '../../data/freeze/BROAD_ml4h_klarqvist___physionet__waveforms__409faaa082084ae5aef22838e35dae06__combined.h5'

In [ ]:
meta_data = meta_data[~meta_data.Age.isna()]

In [ ]:
train_data = meta_data[meta_data['is_graded_train'] == True]
test_data = meta_data[meta_data['is_graded_test'] == True]
validation_data = meta_data[meta_data['is_graded_validation'] == True]

train_ds = ecg_dataset.EcgDataset(h5py_path, train_data.index.values, train_data.Age)
test_ds = ecg_dataset.EcgDataset(h5py_path, test_data.index.values, test_data.Age)
validation_ds = ecg_dataset.EcgDataset(h5py_path, validation_data.index.values, validation_data.Age)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0
    preds = []
    labs = []

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        labs.append(y.flatten().float().numpy())
        y = y.flatten().float().to(device).unsqueeze(-1)
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()
        preds.append(pred.cpu().detach().numpy())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    train_loss /= num_batches
    preds = np.squeeze(np.concatenate(preds))
    labs = np.concatenate(labs)

    print(f"Train Error: \nCorrelation: {(np.corrcoef(preds, labs)[0][1]):>0.5f}, Avg loss: {train_loss:>8f} \n")

def test_loop(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    test_loss = 0
    preds = []
    labs = []

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            labs.append(y.flatten().float().numpy())
            y = y.flatten().float().to(device).unsqueeze(-1)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            preds.append(pred.cpu().detach().numpy())

    test_loss /= num_batches
    preds = np.squeeze(np.concatenate(preds))
    labs = np.concatenate(labs)
    
    print(f"Test Error: \nCorrelation: {(np.corrcoef(preds, labs)[0][1]):>0.5f}, Avg loss: {test_loss:>8f} \n")

In [ ]:
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_dataloader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
val_dataloader   = DataLoader(validation_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done!")

In [ ]:
# 62 min 50.4 sec

In [ ]:
def validation_loop(dataloader, model):
    preds = []
    labs = []
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            labs.append(y.float().numpy())
            y = y.float().to(device).unsqueeze(-1)
            pred = model(X)
            preds.append(pred.cpu().detach().numpy())
            
    return preds, labs

In [ ]:
preds, labs = validation_loop(val_dataloader, model)

In [ ]:
pred_df = pd.DataFrame({'predictions': np.squeeze(np.concatenate(preds)), 'labels': np.concatenate(labs)})
pred_df

In [ ]:
np.corrcoef(pred_df.predictions, pred_df.labels)[0][1]
# 0.7212988866967599

In [ ]:
np.mean(np.abs(pred_df.predictions - pred_df.labels))
# 9.410714

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(pred_df.predictions, pred_df.labels)